# Data Mining Lab 2
#### Team 2:  Patricia Goresen, Jeffrey Lancon, Brychan Manry, George Sturrock

## Source Data
The source of baseball data for this lab is the Sean Lahman Baseball Database [ http://www.seanlahman.com/baseball-archive/statistics/]. Often cited as the most complete baseball database, the data set includes twenty-seven data tables and millions of records covering most non-proprietary baseball data pertaining to offense, fielding, pitching, payroll, player demographics, team statistics, manager data and much more. In past assignments, this team has focused on team level data.  For this Lab, the team will focus on player level data.  In particular, the focus will be on position players.  Pitchers will not be viewed in this study.  This will allow for more exploration of baseball data and provide a broader variety of regression and classification scenarios.  

This lab will focus on the analysis of four specific tables containing player level details in the Lahman Baseball Database:  Batting, Fielding, Player Awards and Salaries.  The names of the different tables accurately describe the contents.  For example, the Batting table contains offensive statics per year for each player.  The tables are at different grains.  The utlimate intention is to have a single dataframe accurately combining all relevant features of the four tables.  The methods used to achieve this goal will be presented below.  To begin, a simple SQL query was written to join the four tables together.  This is a task efficiently handled by any database with few lines of code when compared to the code needed for the same operation in python.  The SQL is shown below:

SELECT Batting.*, AwardsPlayers.*, Fielding.*, Salaries.*
FROM ((Batting LEFT JOIN AwardsPlayers ON (Batting.yearID = AwardsPlayers.yearID) AND (Batting.playerID = AwardsPlayers.playerID)) LEFT JOIN Fielding ON (Batting.stint = Fielding.stint) AND (Batting.yearID = Fielding.yearID) AND (Batting.playerID = Fielding.playerID)) LEFT JOIN Salaries ON (Batting.yearID = Salaries.yearID) AND (Batting.playerID = Salaries.playerID);

The remaining filtering, feature reduction and subsetting tasks will be performed in Python.  


## Import Data

In [19]:
import pandas as pd
import numpy as np

p = pd.read_csv('~/7331_Lab2/data/AwardsPlayers.csv')

print('Awards', awards.info())
print('Batting', batting.info())
print('Fielding', fielding.info())
print('Salaries', salaries.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6158 entries, 0 to 6157
Data columns (total 6 columns):
playerID    6158 non-null object
awardID     6158 non-null object
yearID      6158 non-null int64
lgID        6158 non-null object
tie         47 non-null object
notes       4710 non-null object
dtypes: int64(1), object(5)
memory usage: 288.7+ KB
Awards None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104324 entries, 0 to 104323
Data columns (total 24 columns):
playerID     104324 non-null object
yearID       104324 non-null int64
stint        104324 non-null int64
teamID       104324 non-null object
lgID         103586 non-null object
G            104324 non-null int64
G_batting    0 non-null float64
AB           104324 non-null int64
R            104324 non-null int64
H            104324 non-null int64
2B           104324 non-null int64
3B           104324 non-null int64
HR           104324 non-null int64
RBI          103568 non-null float64
SB           101956 non-null flo

In [20]:
## Go back and convert to table ##
#teamNullCols = pd.DataFrame({
#    'n Missing' : teamNullCols, '% Missing' : ((teamNullCols / teams2.shape[0])*100).round(2)#.astype(str) + "%"})

print("Shape of DataFrames:", "Awards:", awards.shape, "Batting:", batting.shape, "Fielding:", fielding.shape, "Salaries:", salaries.shape)

Shape of DataFrames: Awards: (6158, 6) Batting: (104324, 24) Fielding: (138838, 18) Salaries: (26428, 5)


## Row Filtering

Only records from the 1985 season and greater will be used for analysis in this assignment.  1985 was chosen for two reasons.  First, salary data only goes back to 1985.  Second, the evolution of the game and it players has changed quite a bit since beginning of the professional game of baseball.  Selecting only records from the 1985 season and forward providees a 32 year data set while eliminating data that would likely lead to erroneous results.  

In [22]:
awards = awards[awards.yearID > 1984]
batting = batting[batting.yearID > 1984]
fielding = fielding[fielding.yearID > 1984]
salaries = salaries[salaries.yearID > 1984]

print("Shape of DataFrames:", "Awards:", awards.shape, "Batting:", batting.shape, "Fielding:", fielding.shape, "Salaries:", salaries.shape)

Shape of DataFrames: Awards: (2398, 6) Batting: (42071, 24) Fielding: (54560, 18) Salaries: (26428, 5)


Additionally, players who did not play in at least 25% of the games in a season